In [48]:
from scipy.stats import norm
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from matplotlib.gridspec import GridSpec


# formata float como % com 2 casas decimais
def percentify(val):
    return f'{100*val:.2f}%'

# Discretização do eixo X
xs = np.arange(0,12,.01)

def plot_pdfs(mu_positivo, sigma_positivo, mu_negativo, sigma_negativo, prop, threshold):
  
    threshold_index = np.where(xs>threshold)[0][0]
    fig = plt.figure(figsize=(22,5))
    gs = GridSpec(1, 2, width_ratios=[8, 4])
    plot = fig.add_subplot(gs[0])
    matrix = fig.add_subplot(gs[1])
    
    # valores da PDF para os DOENTES
    y1s = norm.pdf(xs, loc=mu_positivo, scale=sigma_positivo)*prop
    plot.plot(xs, y1s)
    # hachurar os falsos negativos
    error1_vals = np.array((list([0.0]*threshold_index) + list(y1s[threshold_index:])))
    plot.fill_between(xs, error1_vals, alpha=0.5)
    
    # valores da PDF para os NÃO DOENTES
    y2s = norm.pdf(xs, loc=mu_negativo, scale=sigma_negativo)*(1-prop)
    plot.plot(xs, y2s)
    # hachurar os falsos positivos
    error2_vals = np.array((list(y2s[:threshold_index]) + list([0.0]*(len(xs)-threshold_index))))
    plot.fill_between(xs, error2_vals, alpha=0.5)
    
    # linha vertical de limiar/threshold
    plot.axvline(threshold, 0, 1, color='black', alpha=200)
    
    # legendas
    plot.legend(labels=[
        "Distribuição positivos (DOENTE)",
        "Distribuição negativos (NÃO DOENTE)",
        "Threshold ou Limiar [positivos | negativos]",
        "Prob. falsos negativos",
        "Prob. falsos positivos",
    ])

    # fig.title("Distribuições das respostas ao teste")
    
    # Cálculo das probabilidades
    P_vp = norm.cdf(xs[threshold_index], loc=mu_positivo, scale=sigma_positivo)*prop      # verdadeiro positivo
    P_fn = norm.sf(xs[threshold_index], loc=mu_positivo, scale=sigma_positivo)*prop       # falso positivo
    P_vn = norm.sf(xs[threshold_index], loc=mu_negativo, scale=sigma_negativo)*(1-prop)   # verdadeiro negativo
    P_fp = norm.cdf(xs[threshold_index], loc=mu_negativo, scale=sigma_negativo)*(1-prop)  # falso negativo
    
    # Criando matriz de confusão
    df_conf = pd.DataFrame()
    df_conf['Teste negativo'] = [P_vn,P_fn]
    df_conf['Teste positivo'] = [P_fp,P_vp]
    df_conf['idx'] = ['Não doente','Doente']
    df_conf.set_index('idx',inplace=True)

    sns.heatmap(df_conf, annot=True, ax=matrix, cmap="YlGnBu")

    # Construção da matriz de confusão
    d = [
        {'Real':'Positivo', 'Teste positivo':percentify(P_vp), 'Teste negativo':percentify(P_fn), '  % da pop.':percentify(P_vp+P_fn)},
        {'Real':'Negativo', 'Teste positivo':percentify(P_fp), 'Teste negativo':percentify(P_vn), '  % da pop.':percentify(P_fp+P_vn)},
        {'Real':'% da pop.', 'Teste positivo':percentify(P_vp+P_fp), 'Teste negativo':percentify(P_fn+P_vn), '  % da pop.':percentify(P_vp+P_fn+P_fp+P_vn)}
    ]
    print(pd.DataFrame(d).set_index('Real'))
    print('-'*100)
    
    # Cálculo dos indicadores de qualidade do teste
    sensitivity = P_vp/(P_vp+P_fn)    # prob. de classificar corretamente alguém como positivo
    specificity = P_vn/(P_vn+P_fp)    # prob. de classificar corretamente alguém como negativo
    PPV = P_vp/(P_vp+P_fp)            # prob. de ser positivo dado que o teste deu positivo
    NPV = P_vn/(P_vn+P_fn)            # prob. de ser negativo dado que o teste deu negativo
    print(f"Precision/PPV: {100*PPV:.2f}%   NPV: {100*NPV:.2f}%   sensitivity: {100*sensitivity:.2f}%   specificity: {100*specificity:.2f}%")
    
# Configurando sliders interativos
interact(
    plot_pdfs,
    mu_positivo=widgets.IntSlider(min=0, max=10, step=1, value=3),
    sigma_positivo=widgets.FloatSlider(min=0.1, max=5, step=.1, value=1.0),
    mu_negativo=widgets.IntSlider(min=0, max=10, step=1, value=7),
    sigma_negativo=widgets.FloatSlider(min=0.1, max=5, step=.1, value=1.0),
    prop=widgets.FloatSlider(min=0.01, max=0.99, step=.01, value=0.5),
    threshold=widgets.FloatSlider(min=0, max=10, step=.25, value=5),
);


interactive(children=(IntSlider(value=3, description='mu_positivo', max=10), FloatSlider(value=1.0, descriptio…

- ***Precision / PPV*** (Positive Predictive Value) is the percentage of patients with a positive test who actually have the disease
$$Precision = PPV = \frac{true positive}{true positive + false positive}$$
- ***NPV*** (Negative Predictive Value) is the percentage of patients with a negative test who do not have the disease
$$NPV = \frac{true negative}{false negative + true negative}$$
- ***Specificity*** is the ability of a test to correctly classify an individual as disease-free
$$Specificity  = \frac{true negative}{true negative + false positive}$$
- ***Sensitivity*** is the ability of a test to correctly classify an individual as ′diseased′ 
$$Sensitivity = \frac{true positive}{true positive + false negative}$$
> [fonte: Understanding and using sensitivity, specificity and predictive values](https://www.delftstack.com/howto/seaborn/legend-seaborn-plot/)

### Obs:
- Mudar a proporção entre `DOENTES` e `NÃO DOENTES` afeta `PPV` e `NPV`, mas **não** afeta `Specificity` e `Sensitivity`
- Mesmo deixando tudo simétrico, pode ser que os valores fiquem com uma pequena diferença causada pela discretização grosseira do eixo X